In [83]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from common.bio.amino_acid import *
pd.set_option('display.max_colwidth', -1)

## Importing original uniprot file

In [79]:
#uniprot = pd.read_csv("../data/protein/cgan/data_sources/uniprot_all_not-hetero.tab", sep="\t").drop(["Entry","Entry name","Status", "Subunit structure [CC]"], axis=1)
uniprot = pd.read_csv("../data/protein/cgan/data_sources/uniprot_all_not-hetero_dropped.tab", sep="\t").drop(["Unnamed: 0"], axis=1)

In [80]:
uniprot.head()

,Catalytic activity,EC number,Sequence
0,"CATALYTIC ACTIVITY: 3'-phosphoadenylyl sulfate + [heparan sulfate]-glucosamine = adenosine 3',5'-bisphosphate + [heparan sulfate]-N-sulfoglucosamine.",2.8.2.8; 3.-.-.-; 2.8.2.-,MSFIMKPHRHFQRTLILLATFCMVSIIISAYYLYSGYKQESEVSGRASEVDCGDLQHIPSRLMEVRRTMISDASRTDPTVLVFVESQYSSLGQDIIMMLESIRFHYHTEIAPGKGDLPALTDNVKGKYVLIIYENILKYINMDSWNRSLLDKYCIEYGVGIIGFHKTSEKNLQSFQFRGFPFSISGNLAVKDCCINPHSPLLRVTKSSKLDRGSLPGTDWTVFQINHSTYQPVIFAKVKTPENLSPPISKHAFYATIIHDLGLHDGIQRVLFGNNLNFWLHKLIFIDAISFLSGKRLTLSLDRYILVDIDDIFVGKEGTRMNTNDVKALLDTQNLLRTQITNFTFNLGFSGKFYHTGTEEEDEGDDCLLGSVDEFWWFPHMWSHMQPHLFHNESSLIEQMILNKKFALEHGIPTDMGYAVSPHHSGVYPVHVQLYEAWKKVWNIKITSTEEYPHLKPARYRRGFIHKNIMVLPRQTCGLFTHTIFYKEYPGGPRELDKSIHGGELFFTVVLNPISIFMTHLSNYGNDRLGLYTFVNLANFVQTWTNLRLQTLPPAQLAHKYFELFPDQKDPLWQNPCDDKRHRDIWSKEKTCDRLPKFLVIGPQKTGTTALCLFLIMHPSILSNSPSPKSFEEVQFFNRNNYHRGIDWYMDFFPVPSNVTTDFLFEKSANYFHSEDAPKRAASLVPKAKIITILIDPSDRAYSWYQHQRSHEDPAALKFSFYEVISAGPNAPWELRTLQKRCLVPGWYANHIERWLVYFPPFQLLIIDGQHLRTTPATVMDEVQKFLGVSPHYNYSEALTFDSHKGFWCQLLEEGKTKCLGKSKGRKYPPMDSDSRAFLSSYYRDHNVELSKLLHRLGQPLPSWLRQELQKVR
1,CATALYTIC ACTIVITY: ATP + a protein = ADP + a phosphoprotein.,2.7.11.1,MMYLPYAQLPDPTDKFEIYEEIAQGVNAKVFRAKELDNDRIVALKIQHYDEEHQVSIEEEYRTLRDYCDHPNLPEFYGVYKLSKPNGPDEIWFVMEYCAGGTAVDMVNKLLKLDRRMREEHIAYIIRETCRAAIELNRNHVLHRDIRGDNILLTKNGRVKLCDFGLSRQVDSTLGKRGTCIGSPCWMAPEVVSAMESREPDITVRADVWALGITTIELADGKPPFADMHPTRAMFQIIRNPPPTLMRPTNWSQQINDFISESLEKNAENRPMMVEMVEHPFLTELIENEDEMRSDIAEMLELSRDVKTLYKEPELFVDRGYVKRFDEKPEKMYPEDLAALENPVDENIIESLRHRILMGESYSFIGDILLSLNSNEIKQEFPQEFHAKYRFKSRSENQPHIFSVADIAYQDMLHHKEPQHIVLSGESYSGKSTNARLLIKHLCYLGDGNRGATGRVESSIKAILMLVNAGTPVNNDSTRCVLQYCLTFGKTGKMSGAVFNMYMLEKLRVATTDGTQHNFHIFYYFYDFINQQNQLKEYNLKADRNYRYLRVPPEVPPSKLKYRRDDPEGNVERYREFENILRDIDFNHKQLETVRKVLAAILNIGNIRFRQNGKYAEVENTDIVSRIAELLRVDEKKFMWSLTNFIMVKGGIAERRQYTTEEARDARDAVASTLYSRLVDFIINRINMNMSFPRAVFGDTNAIIIHDMFGFECFNRNGLEQLMINTLNEQMQYHYNQRIFISEMLEMEAEDIDTINLNFYDNKTALDNLLTKPDGLFYIIDDASRSCQDQDLIMDRVSEKHSQFVKKHTATEISVAHYTGRIIYDTRAFTDINRDFVPPEMIETFRSSLDESIMLMFTNQLTKAGNLTMPFEAVQHKDESERKSYALNTLSAGCISQVNNLRTLAANFRFTCLTLLKMLSQNANLGVHFVRCIRADLEYKPRSFHSDVVQQQMKALGVLDTVIARQKGFSSRLPFDEFLRRYQFLAFDFDEPVEMTKDNCRLLFLRLKMEGWALGKTKVFLRYYNDEFLARLYELQVKKVIKVQSMMRALLARKRVKGGKVFKLGKKGPEHHDVAASKIQKAFRGFRDRVRLPPLVNEKSGQLNENTADFIRPFAKKWREKSIFQVLLHYRAARFQDFVNLSQQVHIYNQRMVAGLNKCTRAVPFERINMREVNSSQLGPLPVPIKKMPFRLDQIPFYDTQYMVDPANSISRQAFPNQLLTQHMEDDEPWDSPLQRNPSMTSCALTYNAYKKEQACQTNWDRMGESDNIYNQGYFRDPQQLRRNQMQMNMNAYNNAYNSYNSNYNNQNWGVHRSGSRRNSLKGYAAPPPPPPPMPSSNYYRNNPNQQQRNYQQRSSYPPSDPVRELQNMARNEGDNSEDPPFNFKAMLRKTNYPRGSETNTYDFNNRRGSDSGDQHTFQPPKLRSTGRRYQDDEGYNSSSGNYGVSRKFGQQQRAPTLRQSPASVGRSFEDSNARSFEEAGSYVEEEIAPGITLSGYAVDI
2,"CATALYTIC ACTIVITY: NH(3) + 2 H(2)O + 6 ferricytochrome c = nitrite + 6 ferrocytochrome c + 7 H(+). {ECO:0000269|PubMed:16500161, ECO:0000269|PubMed:22281743}.",1.7.2.2,MNDLNRLGRVGRWIAGAACLFLASAAHAEPGENLKPVDAMQCFDCHTQIEDMHTVGKHATVNCVHCHDATEHVETASSRRMGERPVTRMDLEACATCHTAQFNSFVEVRHESHPRLEKATPTSRSPMFDKLIAGHGFAFEHAEPRSHAFMLVDHFVVDRAYGGRFQFKNWQKVTDGMGAVRGAWTVLTDADPESSDQRRFLSQTATAANPVCLNCKTQDHILDWAYMGDEHEAAKWSRTSEVVEFARDLNHPLNCFMCHDPHSAGPRVVRDGLINAVVDRGLGTYPHDPVKSEQQGMTKVTFQRGREDFRAIGLLDTADSNVMCAQCHVEYNCNPGYQLSDGSRVGMDDRRANHFFWANVFDYKEAAQEIDFFDFRHATTGAALPKLQHPEAETFWGSVHERNGVACADCHMPKVQLENGKVYTSHSQRTPRDMMGQACLNCHAEWTEDQALYAIDYIKNYTHGKIVKSEYWLAKMIDLFPVAKRAGVSEDVLNQARELHYDAHLYWEWWTAENSVGFHNPDQARESLMTSISKSKEAVSLLNDAIDAQVASR
3,CATALYTIC ACTIVITY: A monoamide of a dicarboxylate + H(2)O = a dicarboxylate + NH(3).,3.5.1.3,MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFNSPYGAKYFPEYAEKIPGESTQKLSEVAKECSIYLIGGSIPEEDAGKLYNTCAVFGPDGTLLAKYRKIHLFDIDVPGKITFQESKTLSPGDSFSTFDTPYCRVGLGICYDMRFAELAQIYAQRGCQLLVYPGAFNLTTGPAHWELLQRSRAVDNQVYVATASPARDDKASYVAWGHSTVVNPWGEVLAKAGTEEAIVYSDIDLKKLAEIRQQIPVFRQKRSDLYAVEMKKP
4,"CATALYTIC ACTIVITY: ATP + nicotinamide ribonucleotide = diphosphate + NAD(+). {ECO:0000269|PubMed:10428462, ECO:0000269|PubMed:18098602, ECO:0000269|PubMed:3013296}.; CATALYTIC ACTIVITY: ATP + beta-nicotinate-D-ribonucleotide = di

Parsing file that contains all enzyme reactions. Note: Only taking a first reaction for now

In [160]:
f = open("../data/protein/cgan/data_sources/enzyme.dat", "r")
current_id = None
dictionary = {}
for line in f:
    if line.startswith("CA") or line.startswith("ID"):
        components = line.split("   ")
        if components[0] == "ID":
            current_id = components[1].replace("\n", "")
            skip=False
        else:
            reaction = components[1]
            if not current_id in dictionary:
                dictionary[current_id] = components[1].replace("\n", "").replace(".", "").replace("(1) ", "")
            elif components[1].startswith("(2) "):
                skip = True
            elif not skip:
                dictionary[current_id] += " "+ components[1].replace("\n", "").replace(".", "")
                
with open('../data/protein/cgan/data_sources/enzyme_class_reaction.csv', 'w') as f:
    for key in dictionary.keys():
        f.write("%s\t%s\n"%(key,dictionary[key]))
    

## Cleaning reactions

In [84]:
uniprot = filter_non_standard_amino_acids(uniprot, "Sequence")

In [85]:
uniprot.shape

(3906270, 3)

In [86]:
uniprot = uniprot[uniprot.Sequence.str.len() <=1024]

In [87]:
uniprot.shape

(3791448, 3)

In [88]:
uniprot.to_csv("../data/protein/cgan/data_sources/uniprot_all_not-hetero_dropped.tab", sep="\t", index=None)

In [89]:
uniprot.head()

,Catalytic activity,EC number,Sequence
0,"CATALYTIC ACTIVITY: 3'-phosphoadenylyl sulfate + [heparan sulfate]-glucosamine = adenosine 3',5'-bisphosphate + [heparan sulfate]-N-sulfoglucosamine.",2.8.2.8; 3.-.-.-; 2.8.2.-,MSFIMKPHRHFQRTLILLATFCMVSIIISAYYLYSGYKQESEVSGRASEVDCGDLQHIPSRLMEVRRTMISDASRTDPTVLVFVESQYSSLGQDIIMMLESIRFHYHTEIAPGKGDLPALTDNVKGKYVLIIYENILKYINMDSWNRSLLDKYCIEYGVGIIGFHKTSEKNLQSFQFRGFPFSISGNLAVKDCCINPHSPLLRVTKSSKLDRGSLPGTDWTVFQINHSTYQPVIFAKVKTPENLSPPISKHAFYATIIHDLGLHDGIQRVLFGNNLNFWLHKLIFIDAISFLSGKRLTLSLDRYILVDIDDIFVGKEGTRMNTNDVKALLDTQNLLRTQITNFTFNLGFSGKFYHTGTEEEDEGDDCLLGSVDEFWWFPHMWSHMQPHLFHNESSLIEQMILNKKFALEHGIPTDMGYAVSPHHSGVYPVHVQLYEAWKKVWNIKITSTEEYPHLKPARYRRGFIHKNIMVLPRQTCGLFTHTIFYKEYPGGPRELDKSIHGGELFFTVVLNPISIFMTHLSNYGNDRLGLYTFVNLANFVQTWTNLRLQTLPPAQLAHKYFELFPDQKDPLWQNPCDDKRHRDIWSKEKTCDRLPKFLVIGPQKTGTTALCLFLIMHPSILSNSPSPKSFEEVQFFNRNNYHRGIDWYMDFFPVPSNVTTDFLFEKSANYFHSEDAPKRAASLVPKAKIITILIDPSDRAYSWYQHQRSHEDPAALKFSFYEVISAGPNAPWELRTLQKRCLVPGWYANHIERWLVYFPPFQLLIIDGQHLRTTPATVMDEVQKFLGVSPHYNYSEALTFDSHKGFWCQLLEEGKTKCLGKSKGRKYPPMDSDSRAFLSSYYRDHNVELSKLLHRLGQPLPSWLRQELQKVR
2,"CATALYTIC ACTIVITY: NH(3) + 2 H(2)O + 6 ferricytochrome c = nitrite + 6 ferrocytochrome c + 7 H(+). {ECO:0000269|PubMed:16500161, ECO:0000269|PubMed:22281743}.",1.7.2.2,MNDLNRLGRVGRWIAGAACLFLASAAHAEPGENLKPVDAMQCFDCHTQIEDMHTVGKHATVNCVHCHDATEHVETASSRRMGERPVTRMDLEACATCHTAQFNSFVEVRHESHPRLEKATPTSRSPMFDKLIAGHGFAFEHAEPRSHAFMLVDHFVVDRAYGGRFQFKNWQKVTDGMGAVRGAWTVLTDADPESSDQRRFLSQTATAANPVCLNCKTQDHILDWAYMGDEHEAAKWSRTSEVVEFARDLNHPLNCFMCHDPHSAGPRVVRDGLINAVVDRGLGTYPHDPVKSEQQGMTKVTFQRGREDFRAIGLLDTADSNVMCAQCHVEYNCNPGYQLSDGSRVGMDDRRANHFFWANVFDYKEAAQEIDFFDFRHATTGAALPKLQHPEAETFWGSVHERNGVACADCHMPKVQLENGKVYTSHSQRTPRDMMGQACLNCHAEWTEDQALYAIDYIKNYTHGKIVKSEYWLAKMIDLFPVAKRAGVSEDVLNQARELHYDAHLYWEWWTAENSVGFHNPDQARESLMTSISKSKEAVSLLNDAIDAQVASR
3,CATALYTIC ACTIVITY: A monoamide of a dicarboxylate + H(2)O = a dicarboxylate + NH(3).,3.5.1.3,MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFNSPYGAKYFPEYAEKIPGESTQKLSEVAKECSIYLIGGSIPEEDAGKLYNTCAVFGPDGTLLAKYRKIHLFDIDVPGKITFQESKTLSPGDSFSTFDTPYCRVGLGICYDMRFAELAQIYAQRGCQLLVYPGAFNLTTGPAHWELLQRSRAVDNQVYVATASPARDDKASYVAWGHSTVVNPWGEVLAKAGTEEAIVYSDIDLKKLAEIRQQIPVFRQKRSDLYAVEMKKP
4,"CATALYTIC ACTIVITY: ATP + nicotinamide ribonucleotide = diphosphate + NAD(+). {ECO:0000269|PubMed:10428462, ECO:0000269|PubMed:18098602, ECO:0000269|PubMed:3013296}.; CATALYTIC ACTIVITY: ATP + beta-nicotinate-D-ribonucleotide = diphosphate + deamido-NAD(+). {ECO:0000305|PubMed:11884393}.",2.7.7.1; 2.7.7.18,MDPTRAPDFKPPSADEELIPPPDPESKIPKSIPIIPYVLADANSSIDAPFNIKRKKKHPKHHHHHHHSRKEGNDKKHQHIPLNQDDFQPLSAEVSSEDDDADFRSKERYGSDSTTESETRGVQKYQIADLEEVPHGIVRQARTLEDYEFPSHRLSKKLLDPNKLPLVIVACGSFSPITYLHLRMFEMALDAISEQTRFEVIGGYYSPVSDNYQKQGLAPSYHRVRMCELACERTSSWLMVDAWESLQPSYTRTAKVLDHFNHEINIKRGGVATVTGEKIGVKIMLLAGGDLIESMGEPNVWADADLHHILGNYGCLIVERTGSDVRSFLLSHDIMYEHRRNILIIKQLIYNDISSTKVRLFIRRAMSVQYLLPNSVIRYIQEHRLYVDQTEPVKQVLGNKE
5,"CATALYTIC ACTIVITY: ATP + [protein]-L-tyrosine = diphosphate + [protein]-O(4)-(5'-adenylyl)-L-tyrosine. {ECO:0000269|PubMed:22266942, ECO:0000269|PubMed:23738009}.; CATALYTIC ACTIVITY: ATP + [protein]-L-threonine = diphosphate + [protein]-O(4)-(5'-adenylyl)-L-threonine. {ECO:0000269|PubMed:22266942, ECO:0000269|PubMed:23738009}.",2.7.7.n1,MPSENPIGKTMKSIDEQSLHNARRLFESGDIDRIEVGTTAGLQQIHRYLFGGLYDFAGQIREDNISKGGFRFANAMYLKEALVKIEQMPERTFEEIIAKYVEMNIAHPFLEGNGRSTRIWLDLVLKKNLKKVVNWQNVSKTLYLQAMERSPVNDLELRFLLKDNLTDDVDNREIIFKGIEQSYYYEGYEKG


In [100]:
uniprot[["Catalytic activity", "EC number"]].drop_duplicates().sort_values("EC number")

,Catalytic activity,EC number
21842,CATALYTIC ACTIVITY: An oxidized nitroaromatic compound + NADPH = a reduced nitroaromatic compound + NADP(+). {ECO:0000250|UniProtKB:P17117}.,1.-.-.-
21625,CATALYTIC ACTIVITY: NADH + acceptor = NAD(+) + reduced acceptor.,1.-.-.-
959,CATALYTIC ACTIVITY: An oxidized nitroaromatic compound + NADPH = a reduced nitroaromatic compound + NADP(+). {ECO:0000269|PubMed:8755878}.,1.-.-.-
314430,"CATALYTIC ACTIVITY: Myo-inositol + NAD(+) = 2,4,6/3,5-pentahydroxycyclohexanone + NADH. {ECO:0000256|SAAS:SAAS00335677}.",1.-.-.-; 1.1.1.-; 1.1.1.18
217867,CATALYTIC ACTIVITY: 4-phospho-D-erythronate + NAD(+) = (3R)-3-hydroxy-2-oxo-4-phosphonooxybutanoate + NADH. {ECO:0000256|SAAS:SAAS00897192}.,1.-.-.-; 1.1.1.-; 1.1.1.290
70516,"CATALYTIC ACTIVITY: 17-alpha,20-alpha-dihydroxypregn-4-en-3-one + NAD(P)(+) = 17-alpha-hydroxyprogesterone + NAD(P)H. {ECO:0000250|UniProtKB:Q04828}.; CATALYTIC ACTIVITY: Indan-1-ol + NAD(P)(+) = indanone + NAD(P)H. {ECO:0000250|UniProtKB:Q04828}.",1.-.-.-; 1.1.1.149; 1.1.1.112
231522,CATALYTIC ACTIVITY: 4-phospho-D-erythronate + NAD(+) = (3R)-3-hydroxy-2-oxo-4-phosphonooxybutanoate + NADH. {ECO:0000256|SAAS:SAAS00897192}.,1.-.-.-; 1.1.1.290
21843,"CATALYTIC ACTIVITY: A 5,6,7,8-tetrahydropteridine + NAD(P)(+) = a 6,7-dihydropteridine + NAD(P)H.",1.-.-.-; 1.5.1.34
92436,"CATALYTIC ACTIVITY: (S)-lactate + an oxidized electron acceptor = pyruvate + a reduced electron acceptor. {ECO:0000255|HAMAP-Rule:MF_01559, ECO:0000269|PubMed:18473}.",1.1.-.-
208086,"CATALYTIC ACTIVITY: (S)-lactate + an oxidized electron acceptor = pyruvate + a reduced electron acceptor. {ECO:0000256|HAMAP-Rule:MF_01559, ECO:0000256|SAAS:SAAS00848086}.",1.1.-.-


## Splitting reaction into components

Getting substrates and product compounds

In [161]:
enzyme_class_reaction = pd.read_csv("../data/protein/cgan/data_sources/enzyme_class_reaction.csv", sep="\t", header=None, names=["EC class", "Reaction"])

In [162]:
substrates = enzyme_class_reaction.Reaction.str.split("\s\=\s",expand=True)[0]
products = enzyme_class_reaction.Reaction.str.split("\s\=\s",expand=True)[1]

In [163]:
product_components = products.str.split("\s\+\s",expand=True)
product_components.reset_index(drop=True, inplace=True)
p_columns_names = { index:"product_"+str(index+1) for index in range(8)}
product_components = product_components.rename(index=str, columns=p_columns_names)

In [192]:
for c in p_columns_names.values():
    product_components[c] = product_components[c].str.strip()

Validation

In [193]:
product_components[product_components["product_8"].notnull()].head()

,product_1,product_2,product_3,product_4,product_5,product_6,product_7,product_8
5255,5-hydroxybenzimidazole,5'-deoxyadenosine,L-methionine,formate,formate,NH(3),phosphate,oxidized acceptor


Getting substrate compounds

In [167]:
substrate_components = substrates.str.split("\s\+\s",expand=True)
substrate_components.reset_index(drop=True, inplace=True)
s_columns_names = { index:"substrate_"+str(index+1) for index in range(5)}
substrate_components = substrate_components.rename(index=str, columns=s_columns_names)

In [179]:
for c in s_columns_names.values():
    substrate_components[c] = substrate_components[c].str.strip()

In [194]:
substrate_components[substrate_components["substrate_5"].notnull()].head()

,substrate_1,substrate_2,substrate_3,substrate_4,substrate_5
513,An aromatic aldehyde,NADP(+),AMP,diphosphate,H(2)O
1056,2 oxidized coenzyme F420,2 reduced ferredoxin [iron-sulfur] cluster,CoB,CoM,2 H(+)
1058,2 CO(2),2 reduced ferredoxin [iron-sulfur] cluster,CoB,CoM,2 H(+)
1627,Tryptophan,FADH(2),chloride,O(2),H(+)
1667,Tetracycline,FADH(2),chloride,O(2),H(+)


Joining everything together

In [196]:
substrates_products = pd.concat([substrate_components, product_components], axis=1)
substrates_products.shape

In [198]:
enzyme_class_reaction.reset_index(drop=True, inplace=True)
substrates_products.reset_index(drop=True, inplace=True)

In [200]:
final_to_save = pd.concat([enzyme_class_reaction,substrates_products], axis=1)
final_to_save.shape

In [203]:
final_to_save.head()

,EC class,Reaction,product_1,product_2,product_3,product_4,product_5,product_6,product_7,product_8,substrate_1,substrate_2,substrate_3,substrate_4,substrate_5
0,1.1.1.1,A primary alcohol + NAD(+) = an aldehyde + NADH,an aldehyde,NADH,None,None,None,None,None,None,A primary alcohol,NAD(+),None,None,None
1,1.1.1.2,An alcohol + NADP(+) = an aldehyde + NADPH,an aldehyde,NADPH,None,None,None,None,None,None,An alcohol,NADP(+),None,None,None
2,1.1.1.3,L-homoserine + NAD(P)(+) = L-aspartate 4-semialdehyde + NAD(P)H,L-aspartate 4-semialdehyde,NAD(P)H,None,None,None,None,None,None,L-homoserine,NAD(P)(+),None,None,None
3,1.1.1.4,"(R,R)-butane-2,3-diol + NAD(+) = (R)-acetoin + NADH",(R)-acetoin,NADH,None,None,None,None,None,None,"(R,R)-butane-2,3-diol",NAD(+),None,None,None
4,1.1.1.6,Glycerol + NAD(+) = glycerone + NADH,glycerone,NADH,None,None,None,None,None,None,Glycerol,NAD(+),None,None,None


In [205]:
final_to_save.to_csv("../data/protein/cgan/enzyme_reaction_splitted_in_components.csv", sep="\t")

## ChEBI (initial preprocessing)

Trying to create dictornary for synonyms <-> smiles

In [206]:
from rdkit.Chem import PandasTools

molecules = PandasTools.LoadSDF('../data/ChEBI_complete_3star.sdf',
                              smilesName='SMILES',
                              molColName='Molecule',
                              includeFingerprints=False)

ModuleNotFoundError: No module named 'rdkit'

In [225]:
molecules = molecules[["ChEBI ID","ChEBI Name", "Formulae", "SMILES", "Synonyms", "UniProt Database Links"]]
molecules.head()

,ChEBI ID,ChEBI Name,Formulae,SMILES,Synonyms,UniProt Database Links
0,CHEBI:90,(-)-epicatechin,C15H14O6,Oc1cc(O)c2c(c1)OC(c1ccc(O)c(O)c1)C(O)C2,(-)-Epicatechin,Q9SEV0
1,CHEBI:165,"(1S,4R)-fenchone",C10H16O,CC12CCC(C1)C(C)(C)C2=O,"(+)-2-fenchanone\n(+)-fenchan-2-one\n(+)-fenchone\n(1S)-(+)-fenchone\n(1S)-fenchone\n(1S,4R)-(+)-2-fenchanone\n(1S,4R)-(+)-fenchan-2-one\n(1S,4R)-(+)-fenchone\n(1S,4R)-1,3,3-trimethylbicyclo[2.2.1]heptan-2-one\n(1S,4R)-2-fenchanone\n(1S,4R)-fenchan-2-one\n(1S,4R)-fenchone\nd-fenchone",NaN
2,CHEBI:598,1-alkyl-2-acylglycerol,C4H6O4R2,*OCC(CO)OC(*)=O,1-alkyl-2-acylglycerols\n2-Acyl-1-alkyl-sn-glycerol,NaN
3,CHEBI:776,16alpha-hydroxyestrone,C18H22O3,CC12CCC3c4ccc(O)cc4CCC3C1CC(O)C2=O,"16 alpha OHE\n16alpha-Hydroxyestrone\n16alpha-hydroxyestrone\n3,16alpha-Dihydroxyestra-1,3,5(10)-trien-17-one\n3,16alpha-dihydroxy-1,3,5(10)-estratrien-17-one\nestra-1,3,5(10)-triene-3,16alpha-diol-17-one",NaN
4,CHEBI:943,"2,6-dichlorobenzonitrile",C7H3Cl2N,N#Cc1c(Cl)cccc1Cl,"2,6-DBN\n2,6-Dichlorobenzoic acid nitrile\n2,6-Dichlorobenzonitrile\n2,6-Dichlorophenyl cyanide\nDBN\nDCB\nDichlobanil\ndichlobenil",A2A5R2\nD4A631\nG3X9K3\nO46382\nQ42510\nQ7TSU1\nQ9Y6D5\nQ9Y6D6


In [227]:
molecules.to_csv("../data/ChEBI_select.csv", sep='\t')

## Filtered ChEBI preprocessing

In [265]:
molecules = pd.read_csv("../data/protein/cgan/ChEBI_select.csv", sep='\t')

In [248]:
molecules = molecules.drop("Unnamed: 0", axis=1)
molecules.head()

,ChEBI ID,ChEBI Name,Formulae,SMILES,Synonyms,UniProt Database Links
0,CHEBI:90,(-)-epicatechin,C15H14O6,Oc1cc(O)c2c(c1)OC(c1ccc(O)c(O)c1)C(O)C2,(-)-Epicatechin,Q9SEV0
1,CHEBI:165,"(1S,4R)-fenchone",C10H16O,CC12CCC(C1)C(C)(C)C2=O,"(+)-2-fenchanone\r\n(+)-fenchan-2-one\r\n(+)-fenchone\r\n(1S)-(+)-fenchone\r\n(1S)-fenchone\r\n(1S,4R)-(+)-2-fenchanone\r\n(1S,4R)-(+)-fenchan-2-one\r\n(1S,4R)-(+)-fenchone\r\n(1S,4R)-1,3,3-trimethylbicyclo[2.2.1]heptan-2-one\r\n(1S,4R)-2-fenchanone\r\n(1S,4R)-fenchan-2-one\r\n(1S,4R)-fenchone\r\nd-fenchone",NaN
2,CHEBI:598,1-alkyl-2-acylglycerol,C4H6O4R2,*OCC(CO)OC(*)=O,1-alkyl-2-acylglycerols\r\n2-Acyl-1-alkyl-sn-glycerol,NaN
3,CHEBI:776,16alpha-hydroxyestrone,C18H22O3,CC12CCC3c4ccc(O)cc4CCC3C1CC(O)C2=O,"16 alpha OHE\r\n16alpha-Hydroxyestrone\r\n16alpha-hydroxyestrone\r\n3,16alpha-Dihydroxyestra-1,3,5(10)-trien-17-one\r\n3,16alpha-dihydroxy-1,3,5(10)-estratrien-17-one\r\nestra-1,3,5(10)-triene-3,16alpha-diol-17-one",NaN
4,CHEBI:943,"2,6-dichlorobenzonitrile",C7H3Cl2N,N#Cc1c(Cl)cccc1Cl,"2,6-DBN\r\n2,6-Dichlorobenzoic acid nitrile\r\n2,6-Dichlorobenzonitrile\r\n2,6-Dichlorophenyl cyanide\r\nDBN\r\nDCB\r\nDichlobanil\r\ndichlobenil",A2A5R2\r\nD4A631\r\nG3X9K3\r\nO46382\r\nQ42510\r\nQ7TSU1\r\nQ9Y6D5\r\nQ9Y6D6


In [249]:
molecules.loc[molecules.Synonyms.notnull(), "Synonyms"] = molecules.loc[molecules.Synonyms.notnull(), "Synonyms"] + "\r\n" +molecules.loc[molecules.Synonyms.notnull(), "ChEBI Name"] 

In [250]:
molecules.loc[molecules.Synonyms.isnull(), "Synonyms"] = molecules.loc[molecules.Synonyms.isnull(), "ChEBI Name"] 

In [251]:
molecules[molecules.Synonyms.isnull()].head()

,ChEBI ID,ChEBI Name,Formulae,SMILES,Synonyms,UniProt Database Links


In [252]:
molecules.shape

(45176, 6)

In [253]:
molecules_selected = molecules[["Formulae", "SMILES", "Synonyms"]]

In [266]:
import common.preprocessing
molecules_expanded = split_dataframe_list_to_rows(molecules_selected, "Synonyms", "\r\n")
molecules_expanded.shape

In [267]:
molecules_expanded["Synonyms"] = molecules_expanded["Synonyms"].str.strip()
molecules_expanded["Formulae"] = molecules_expanded["Formulae"].str.strip()
molecules_expanded["SMILES"] = molecules_expanded["SMILES"].str.strip()

In [269]:
molecules_expanded = molecules_expanded.set_index(['Synonyms'])

In [271]:
molecules_expanded.head()

,Formulae,SMILES
Synonyms,,
(-)-Epicatechin,C15H14O6,Oc1cc(O)c2c(c1)OC(c1ccc(O)c(O)c1)C(O)C2
(-)-epicatechin,C15H14O6,Oc1cc(O)c2c(c1)OC(c1ccc(O)c(O)c1)C(O)C2
(+)-2-fenchanone,C10H16O,CC12CCC(C1)C(C)(C)C2=O
(+)-fenchan-2-one,C10H16O,CC12CCC(C1)C(C)(C)C2=O
(+)-fenchone,C10H16O,CC12CCC(C1)C(C)(C)C2=O


In [273]:
molecules_expanded.to_csv("../data/protein/cgan/smiles_dict.csv", sep='\t', quoting=csv.QUOTE_MINIMAL)

## Matching compounds with smiles

Find compouds that cannot be match automatically

In [9]:
import csv
molecules = pd.read_csv("../data/protein/cgan/smiles_dict.csv", sep='\t')
proteins = pd.read_csv("../data/protein/cgan/enzyme_reaction_splitted_in_components.csv", sep="\t")

In [3]:
molecules.shape, proteins.shape

((175903, 3), (6238, 16))

In [4]:
proteins.head(1)

,Unnamed: 0,EC class,Reaction,product_1,product_2,product_3,product_4,product_5,product_6,product_7,product_8,substrate_1,substrate_2,substrate_3,substrate_4,substrate_5
0,0,1.1.1.1,A primary alcohol + NAD(+) = an aldehyde + NADH,an aldehyde,NADH,NaN,NaN,NaN,NaN,NaN,NaN,A primary alcohol,NAD(+),NaN,NaN,NaN


In [11]:
def concatenate(df, columns, all_values = []):
    for c in  columns:
        all_values = np.concatenate((all_values, df[c].astype(str).unique()))
    return all_values

In [12]:
all_uniq = None
all_uniq = concatenate(proteins,  [col for col in proteins.columns if col.startswith("substrate")])
all_uniq = concatenate(proteins,  [col for col in proteins.columns if col.startswith("product")], all_uniq)
all_uniq = all_uniq[all_uniq != 'nan']
all_uniq =  list(set(all_uniq))
len(all_uniq)

8818

In [13]:
i = 0
n_found = 0
results = []
for i in tqdm(range(len(all_uniq))):
    compound = all_uniq[i]
    compound = compound.replace('- ', '-')
    compound = compound.replace('- ','-')
#     compound = re.sub(r'\(\d+\) in tRNA.*$', '', compound)
#     compound = re.sub(r'\(\d+\) in .* rRNA.*$', '', compound)
#     compound = re.sub(r'\(\d+\) in .* rRNA.*$', '', compound)
#     compound = re.sub(r' in .* rRNA.*$', '', compound)
#     compound = re.sub(r' in .* RNA$', '', compound)
#     compound = compound.replace(' in DNA', '')
    matching_smiles = molecules[(molecules['Synonyms'] == compound)].SMILES.values
    if len(matching_smiles) == 0:
        smiles = "Not found"
    else:
        n_found = n_found + 1 
        smiles = matching_smiles[0]
    results.append([all_uniq[i], smiles])
#     if i == 250:
#         break
        
print("Matched compounds: {0:.1%}".format(float(n_found)/i))
print("Not matched: {}".format(i - n_found))

100%|██████████| 8818/8818 [10:33<00:00, 13.92it/s]


Matched compounds: 47.8%
Not matched: 4599


In [325]:
#N(6)-methyladenine

In [396]:
molecules[molecules['Synonyms'].str.lower().str.contains("NADPH")].sort_values("Synonyms").head(100)

,Synonyms,Formulae,SMILES


In [329]:
results

[["chondroitin 4'-sulfate", 'Not found'],
 ['(2S)-2-hydroxy-2- methyl-3-oxobutanoate', 'CC(=O)C(C)(O)C(=O)[O-]'],
 ['2,4,6/3,5- pentahydroxycyclohexanone', 'O=C1C(O)C(O)C(O)C(O)C1O'],
 ['guanine(1405) in 16S rRNA', 'Nc1nc2[nH]cnc2c(=O)[nH]1'],
 ["7,8- dihydroneopterin 3'-phosphate",
  'Nc1nc2c(c(=O)[nH]1)N=C(C(O)C(O)COP(=O)(O)O)CN2'],
 ['Selective cleavage of 98-Arg-|-Ile-99 bond in Limulus proclotting enzyme to form active clotting enzyme',
  'Not found'],
 ['(1R,2S)-1-hydroxybutane-1,2,4- tricarboxylate', 'Not found'],
 ['ginsenoside Rd',
  'CC(C)=CCCC(C)(OC1OC(CO)C(O)C(O)C1O)C1CCC2(C)C1C(O)CC1C3(C)CCC(OC4OC(CO)C(O)C(O)C4OC4OC(CO)C(O)C(O)C4O)C(C)(C)C3CCC12C'],
 ['2-epi-5-epi-valiolone 7-phosphate',
  'O=C1CC(O)(COP(=O)([O-])[O-])C(O)C(O)C1O'],
 ['Diphospho-myo-inositol polyphosphate', 'Not found'],
 ['4 ATP', 'Not found'],
 ['6 oxidized ferredoxin [iron-sulfur] cluster', 'Not found'],
 ['(5Z,8Z,11Z,14Z,17Z)-icosapentaenoate', 'Not found'],
 ["a 5'-half-tRNA molecule with a 2',3'-cycl

In [14]:
import csv
with open("../data/protein/cgan/compound_to_smile.csv", 'w', newline='') as myfile:
    wr = csv.writer(myfile, delimiter='\t')
    wr.writerows(results)

In [10]:
compound_smile_dict = pd.read_csv("../data/protein/cgan/compound_to_smile.csv", sep='\t', header=None, names = ["Compound", "Smiles"])

In [11]:
compound_smile_dict_filtered = compound_smile_dict[compound_smile_dict.Smiles == "Not found"]

In [12]:
compound_smile_list = compound_smile_dict_filtered['Compound'].tolist()

In [13]:
compound_smile_list

['2-dehydro-3-deoxy-D-pentonate',
 '[pyruvate, water dikinase]',
 '[3-methyl-2-oxobutanoate dehydrogenase (acetyl-transferring)] phosphate',
 '[formate C-acetyltransferase]-glycine',
 'nonpolar amino acid(Out)',
 'tRNA pseudouridine(55)',
 '3-lysolecithin',
 'The recombinant enzyme hydrolyzes proteins (serum albumin, collagen) and synthetic substrates (Z-Phe-Arg-NHMec > Z-Leu-Arg-NHMec > Z-Val- Arg-NHMec)',
 'Preferential cleavage: Arg-|-Xaa, including Arg-|-Pro bond, but not Lys-|-Xaa',
 '3 S-adenosyl-L-methionine',
 '4-hydroxy-3-methoxyphenyl-beta-hydroxypropanoyl-CoA',
 'a (2E)-alk-2-enoyl-CoA',
 'an N-terminal-L-methionyl-L-leucyl-[protein]',
 'D-galactosyl-(1->3)-beta-D-galactosyl-(1->4)-beta-D-glucosyl-R',
 '2 Mn(3+)',
 '4 NADH',
 '3-phospho-L-serine',
 'A steroid',
 "a (2'R)- 2'-hydroxyphytoceramide",
 '6 oxidized ferredoxin [iron- sulfur] cluster',
 '[protein]-N(omega)-methyl-L-arginine',
 'Random hydrolysis of (1->6)-linkages in (1->6)-beta-D-glucans',
 '23S rRNA pseudouridine

In [14]:
i = 0
n_found = 0
results = []
for i in tqdm(range(len(compound_smile_list))):
    compound = compound_smile_list[i]
    compound = compound.replace('- ', '-')
    compound = compound.replace('- ','-')
    compound = compound.replace('(Side 1)','')
    compound = compound.replace('(Side 2)','')
#     compound = re.sub('\(\d+\) in \d+S [t,r]RNA$', '', compound)
#     compound = re.sub('\(\d+\) in [t,r]RNA$', '', compound)
    compound = compound.replace('(In)','')
    compound = compound.replace('(Out)','')
    compound = compound.replace(' in DNA', '')
    compound = compound.replace(' in rRNA', '')
    compound = compound.replace(' in tRNA', '')
    compound = re.sub('^[A,a] ', '', compound)
    compound = re.sub('^[A,a]n ', '', compound)
    compound = re.sub('^[\d+] ', '', compound)
    compound = re.sub(' [\d+]$', '', compound)
    compound = re.sub('^n ', '', compound)
    compound = re.sub('^2n ', '', compound)
    compound = compound.replace('(', '')
    compound = compound.replace(')', '')
    matching_smiles = molecules[(molecules['Synonyms'].str.lower() == compound.strip().lower())].SMILES.values
    if len(matching_smiles) == 0:
        smiles = "Not found"
        #print ("Compound: {} \tSmiles: {} | original: {}".format(compound, smiles, original_compound))
    else:
        n_found = n_found + 1 
        smiles = matching_smiles[0]
    #print ("Compound: {} \tSmiles: {}".format(compound, smiles))
        compound_smile_dict.loc[compound_smile_dict.Compound == compound_smile_list[i], "Smiles"] = smiles
        
print("Matched compounds: {0:.1%}".format(float(n_found)/i))
print("Not matched: {}".format(i - n_found))

100%|██████████| 4600/4600 [23:26<00:00,  3.27it/s]


Matched compounds: 24.8%
Not matched: 3458


In [16]:
compound_smile_dict.to_csv("../data/protein/cgan/compound_to_smile.csv", sep='\t', header=True, index=None)

In [200]:
compound_smile_dict = pd.read_csv("../data/protein/cgan/compound_to_smile.csv", sep='\t').drop(["Unnamed: 0"], axis = 1 )

In [201]:
compound_smile_dict_filtered = compound_smile_dict[compound_smile_dict.Smiles == "Not found"]
compound_smile_dict[compound_smile_dict.Smiles == "Not found"].count()

Compound    3459
Smiles      3459
dtype: int64

In [202]:
compound_smile_list = compound_smile_dict_filtered['Compound'].tolist()

In [203]:
hand_collected_smiles = pd.read_csv("../data/protein/cgan/data_sources/missing_components_for_chosen_class-noproteinorrnaEC346.txt", 
                                    sep='\t', header=None, names = ["Component", "SMILES"])

In [205]:
i = 0
n_found = 0
results = []
for i in tqdm(range(len(compound_smile_list))):  
    compound = compound_smile_list[i]
    matching_smiles = hand_collected_smiles[(hand_collected_smiles["Component"].str.strip() == compound.strip())].SMILES.values
    if len(matching_smiles) == 0:
        smiles = "Not found"
        #print ("Compound: {} \tSmiles: {} | original: {}".format(compound, smiles, compound))
    else:
        n_found = n_found + 1 
        smiles = matching_smiles[0]
    #print ("Compound: {} \tSmiles: {}".format(compound, smiles))
        compound_smile_dict.loc[compound_smile_dict.Compound == compound_smile_list[i], "Smiles"] = smiles
print("Matched compounds: {0:.1%}".format(float(n_found)/(i+1)))
print("Not matched: {}".format(i + 1 - n_found))

100%|██████████| 3459/3459 [00:04<00:00, 739.62it/s]


Matched compounds: 6.8%
Not matched: 3224


In [206]:
compound_smile_dict[compound_smile_dict.Smiles == "Not found"].count()

Compound    3224
Smiles      3224
dtype: int64

In [227]:
compound_smile_dict.Smiles = compound_smile_dict.Smiles.str.strip()
compound_smile_dict.Smiles = compound_smile_dict.Smiles.str.replace(" ", "")

In [228]:
compound_smile_dict.to_csv("../data/protein/cgan/compound_to_smile.csv", sep='\t', header=True, index=None)

## Filtering out non common smiles characters

In [229]:
compound_smile_dict = pd.read_csv("../data/protein/cgan/compound_to_smile.csv", sep='\t')

In [230]:
compound_smile_dict = compound_smile_dict[compound_smile_dict.Compound != "Compound"]
compound_smile_dict = compound_smile_dict[compound_smile_dict.Smiles != "Not found"]
compound_smile_dict

,Compound,Smiles
0,2-deoxy-D-glucose,OCC1OC(O)CC(O)C1O
1,"(1R,6R)-1,4,5,5a,6,9-hexahydrophenazine-1,6-dicarboxylate",O=C([O-])C1C=CCC2=C1N=C1CC=CC(C(=O)[O-])C1N2
2,2-dehydro-3-deoxy-D-pentonate,O=C(O)C(=O)CC(O)CO
3,D-tryptophan,NC(Cc1c[nH]c2ccccc12)C(=O)O
4,"7,8- dihydromethanopterin",CC(Nc1ccc(CC(O)C(O)C(O)COC2OC(COP(=O)(O)OC(CCC(=O)O)C(=O)O)C(O)C2O)cc1)C1=Nc2c(nc(N)[nH]c2=O)NC1C
5,8'-apo-beta-carotenol,CC(C=CC=C(C)C=CC1=C(C)CCCC1(C)C)=CC=CC=C(C)C=CC=C(C)CO
6,6-hydroxyprotopine,CN1Cc2c(ccc3c2OCO3)CC(=O)c2cc3c(cc2CC1O)OCO3
7,4-amino-5- hydroxymethyl-2-methylpyrimidine,Cc1ncc(CO)c(N)n1
8,betaine aldehyde,C[N+](C)(C)CC=O
9,thioglycolate,O=C([O-])CS


In [231]:
compound_smile_dict[compound_smile_dict.Smiles.str.contains('%')]

,Compound,Smiles
2045,beta-hematin,C=CC1=C(C)C2=Cc3c(C=C)c(C)c4n3[Fe-2]356OC(=O)CCc7c(C)c8n9c7=CC7=[N+]%10C(=Cc%11c(C)c(C=C)c%12n%11[Fe-2]9%10(OC(=O)CCC9=C(C)C(=C4)[N+]3=C9C=c3c(CCC(=O)[O-])c(C)c(n35)=CC1=[N+]26)[N+]1=C(C=8)C(C=C)=C(C)C1=C%12)C(C)=C7CCC(=O)[O-]


In [232]:
A- "As", B- "Br", K - "K", M - "Mo", "Mg" W - "W" Z - "Zn" a - "Na", "Ca", d - "Cd", e - "Fe", g - "Mg", i - "Ni", l - "Cl",
r - "Br", u - "Cu"

SyntaxError: invalid syntax (<ipython-input-232-d70163be70d1>, line 1)

In [233]:
set_smiles_characters = set([])
[ set_smiles_characters.update(set(val.strip())) for index, val in compound_smile_dict.Smiles.iteritems()]
set_smiles_characters.add(' ')
set_smiles_characters = list(sorted(set_smiles_characters))

In [234]:
common_characters = []
filter_out = []
for item in set_smiles_characters:
    count = compound_smile_dict[compound_smile_dict.Smiles.str.contains(re.escape(item))].count().Compound
    print("{} - count: {}".format(item, count))
    if count >= 100:
        common_characters.append(item)
    else:
        filter_out.append(item)

  - count: 0
# - count: 57
% - count: 1
( - count: 5161
) - count: 5161
* - count: 483
+ - count: 612
- - count: 1749
. - count: 1
0 - count: 2
1 - count: 3765
2 - count: 2454
3 - count: 1483
4 - count: 471
5 - count: 214
6 - count: 78
7 - count: 48
8 - count: 18
9 - count: 12
: - count: 1
= - count: 4777
@ - count: 1
A - count: 11
B - count: 9
C - count: 5223
F - count: 25
H - count: 835
I - count: 14
K - count: 2
M - count: 46
N - count: 2415
O - count: 5160
P - count: 1404
R - count: 5
S - count: 622
T - count: 1
W - count: 2
X - count: 4
Z - count: 2
[ - count: 2262
] - count: 2262
a - count: 10
c - count: 2227
d - count: 2
e - count: 25
g - count: 38
i - count: 8
l - count: 74
n - count: 1253
o - count: 195
p - count: 1
r - count: 10
s - count: 39
t - count: 1
u - count: 2


In [235]:
common_characters.append(" ")
common_characters = sorted(common_characters)

In [237]:
indexToCharacter = {i:common_characters[i] for i in range (len(common_characters))}
characterToIndex = {common_characters[i]:i for i in range (len(common_characters))}
indexToCharacter, characterToIndex

({0: ' ',
  1: '(',
  2: ')',
  3: '*',
  4: '+',
  5: '-',
  6: '1',
  7: '2',
  8: '3',
  9: '4',
  10: '5',
  11: '=',
  12: 'C',
  13: 'H',
  14: 'N',
  15: 'O',
  16: 'P',
  17: 'S',
  18: '[',
  19: ']',
  20: 'c',
  21: 'n',
  22: 'o'},
 {' ': 0,
  '(': 1,
  ')': 2,
  '*': 3,
  '+': 4,
  '-': 5,
  '1': 6,
  '2': 7,
  '3': 8,
  '4': 9,
  '5': 10,
  '=': 11,
  'C': 12,
  'H': 13,
  'N': 14,
  'O': 15,
  'P': 16,
  'S': 17,
  '[': 18,
  ']': 19,
  'c': 20,
  'n': 21,
  'o': 22})

In [238]:
filtered_out = compound_smile_dict
for non_common_character in filter_out:
    filtered_out = filtered_out[~filtered_out.Smiles.str.contains(re.escape(non_common_character))]

In [239]:
filtered_out.shape

(5243, 2)

In [240]:
filtered_out[filtered_out.Smiles.str.contains('%')]

,Compound,Smiles


In [241]:
filtered_out.to_csv("../data/protein/cgan/compound_to_smile_filtered.csv", sep='\t', header=True, index=None)

# Getting ready for CWGAN

In [243]:
#compound_smile_dict = pd.read_csv("../data/protein/cgan/compound_to_smile_filtered.csv", sep='\t') 
compound_smile_dict = pd.read_csv("../data/protein/cgan/compound_to_smile.csv", sep='\t')
proteins = pd.read_csv("../data/protein/cgan/enzyme_reaction_splitted_in_components.csv", sep="\t").drop("Unnamed: 0", axis= 1)

In [244]:
compound_smile_dict.shape, proteins.shape

((5594, 2), (6238, 15))

In [245]:
compound_smile_dict = compound_smile_dict[compound_smile_dict.Compound != "Compound"]
compound_smile_dict = compound_smile_dict[compound_smile_dict.Smiles != "Not found"]
compound_smile_dict.head(10)

,Compound,Smiles
0,2-deoxy-D-glucose,OCC1OC(O)CC(O)C1O
1,"(1R,6R)-1,4,5,5a,6,9-hexahydrophenazine-1,6-dicarboxylate",O=C([O-])C1C=CCC2=C1N=C1CC=CC(C(=O)[O-])C1N2
2,2-dehydro-3-deoxy-D-pentonate,O=C(O)C(=O)CC(O)CO
3,D-tryptophan,NC(Cc1c[nH]c2ccccc12)C(=O)O
4,"7,8- dihydromethanopterin",CC(Nc1ccc(CC(O)C(O)C(O)COC2OC(COP(=O)(O)OC(CCC(=O)O)C(=O)O)C(O)C2O)cc1)C1=Nc2c(nc(N)[nH]c2=O)NC1C
5,8'-apo-beta-carotenol,CC(C=CC=C(C)C=CC1=C(C)CCCC1(C)C)=CC=CC=C(C)C=CC=C(C)CO
6,6-hydroxyprotopine,CN1Cc2c(ccc3c2OCO3)CC(=O)c2cc3c(cc2CC1O)OCO3
7,4-amino-5- hydroxymethyl-2-methylpyrimidine,Cc1ncc(CO)c(N)n1
8,betaine aldehyde,C[N+](C)(C)CC=O
9,thioglycolate,O=C([O-])CS


In [246]:
proteins.head()

,EC class,Reaction,product_1,product_2,product_3,product_4,product_5,product_6,product_7,product_8,substrate_1,substrate_2,substrate_3,substrate_4,substrate_5
0,1.1.1.1,A primary alcohol + NAD(+) = an aldehyde + NADH,an aldehyde,NADH,NaN,NaN,NaN,NaN,NaN,NaN,A primary alcohol,NAD(+),NaN,NaN,NaN
1,1.1.1.2,An alcohol + NADP(+) = an aldehyde + NADPH,an aldehyde,NADPH,NaN,NaN,NaN,NaN,NaN,NaN,An alcohol,NADP(+),NaN,NaN,NaN
2,1.1.1.3,L-homoserine + NAD(P)(+) = L-aspartate 4-semialdehyde + NAD(P)H,L-aspartate 4-semialdehyde,NAD(P)H,NaN,NaN,NaN,NaN,NaN,NaN,L-homoserine,NAD(P)(+),NaN,NaN,NaN
3,1.1.1.4,"(R,R)-butane-2,3-diol + NAD(+) = (R)-acetoin + NADH",(R)-acetoin,NADH,NaN,NaN,NaN,NaN,NaN,NaN,"(R,R)-butane-2,3-diol",NAD(+),NaN,NaN,NaN
4,1.1.1.6,Glycerol + NAD(+) = glycerone + NADH,glycerone,NADH,NaN,NaN,NaN,NaN,NaN,NaN,Glycerol,NAD(+),NaN,NaN,NaN


In [247]:
proteins = proteins[~proteins.product_3.notnull()]
proteins = proteins[~proteins.substrate_3.notnull()]

In [248]:
proteins.shape

(4716, 15)

In [249]:
smiles_columns = ["product_1", "product_2", "substrate_1", "substrate_2"]

In [263]:
proteins_for_joining = proteins[["EC class", *smiles_columns]]
#proteins_for_joining = proteins

In [264]:
proteins_for_joining.head(3)

,EC class,product_1,product_2,substrate_1,substrate_2
0,1.1.1.1,an aldehyde,NADH,A primary alcohol,NAD(+)
1,1.1.1.2,an aldehyde,NADPH,An alcohol,NADP(+)
2,1.1.1.3,L-aspartate 4-semialdehyde,NAD(P)H,L-homoserine,NAD(P)(+)


In [265]:
proteins_for_joining.shape

(4716, 5)

In [266]:
column_list = proteins_for_joining.columns.tolist()
column_list.remove('EC class')
# column_list.remove('Reaction')
column_list

['product_1', 'product_2', 'substrate_1', 'substrate_2']

In [267]:
#for column in smiles_columns:
for column in column_list:
    proteins_for_joining = pd.merge(proteins_for_joining, compound_smile_dict, left_on = column, 
                                    right_on="Compound", how="left").drop("Compound", axis=1)
    proteins_for_joining = proteins_for_joining.rename(columns={"Smiles": "smiles_"+column})
    #proteins_for_joining = proteins_for_joining[proteins_for_joining["smiles_"+column] != "Not found"]

In [268]:
proteins_for_joining.head(3)

,EC class,product_1,product_2,substrate_1,substrate_2,smiles_product_1,smiles_product_2,smiles_substrate_1,smiles_substrate_2
0,1.1.1.1,an aldehyde,NADH,A primary alcohol,NAD(+),*C=O,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,*CO,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
1,1.1.1.2,an aldehyde,NADPH,An alcohol,NADP(+),*C=O,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)C=CC1,CCO,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)c1
2,1.1.1.3,L-aspartate 4-semialdehyde,NAD(P)H,L-homoserine,NAD(P)(+),[NH3+]C(CC=O)C(=O)[O-],NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)C=CC1,NC(CCO)C(=O)O,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)c1


In [147]:
promissing = pd.read_csv("../data/protein/cgan/promissing_classes_all.txt", sep='\t')
missing_components = pd.merge(promissing, proteins_for_joining, left_on = "EC class", 
                                    right_on="EC class", how="inner").set_index("EC class")

missing_components = missing_components[missing_components.product_4.isnull()]
missing_components = missing_components[missing_components.substrate_4.isnull()]

In [148]:
missing_components

,Unnamed: 0,Sequence,Reaction,product_1,product_2,product_3,product_4,product_5,product_6,product_7,...,smiles_product_4,smiles_product_5,smiles_product_6,smiles_product_7,smiles_product_8,smiles_substrate_1,smiles_substrate_2,smiles_substrate_3,smiles_substrate_4,smiles_substrate_5
EC class,,,,,,,,,,,,,,,,,,,,,
3.1.1.1,0,97,A carboxylic ester + H(2)O = an alcohol + a carboxylate,an alcohol,a carboxylate,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,*OC(*)=O,O,NaN,NaN,NaN
3.1.1.10,1,1,Atropine + H(2)O = tropine + tropate,tropine,tropate,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,CN1C2CCC1CC(OC(=O)C(CO)c1ccccc1)C2,O,NaN,NaN,NaN
3.1.1.101,2,1,(Ethylene terephthalate)(n) + H(2)O = (ethylene terephthalate)(n-1) + ethylene terephthalate,(ethylene terephthalate)(n-1),ethylene terephthalate,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Not found,O,NaN,NaN,NaN
3.1.1.102,3,1,Ethylene terephthalate + H(2)O = terephthalate + ethylene glycol,terephthalate,ethylene glycol,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Not found,O,NaN,NaN,NaN
3.1.1.11,4,102,Pectin + n H(2)O = n methanol + pectate,n methanol,pectate,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Not found,O,NaN,NaN,NaN
3.1.1.13,5,12,A steryl ester + H(2)O = a sterol + a fatty acid,a sterol,a fatty acid,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,*C(=O)OC1CCC2(C)C(CCC3C4CCC(*)C4(C)CCC32)C1,O,NaN,NaN,NaN
3.1.1.14,6,6,Chlorophyll + H(2)O = phytol + chlorophyllide,phytol,chlorophyllide,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C=Cc1c(C)c2n3c1C=C1C(C)=C(CC)C4=[N+]1[Mg-2]31n3c(c(C)c5c3=C(C3=[N+]1C(=C2)C(C)C3CCC(=O)OCC=C(C)CCCC(C)CCCC(C)CCCC(C)C)C(C(=O)OC)C5=O)=C4,O,NaN,NaN,NaN
3.1.1.15,7,1,"L-arabinono-1,4-lactone + H(2)O = L-arabinonate",L-arabinonate,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,O=C1OC(CO)C(O)C1O,O,NaN,NaN,NaN
3.1.1.17,8,13,"D-glucono-1,5-lactone + H(2)O = D-gluconate",D-gluconate,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,O=C1OC(CO)C(O)C(O)C1O,O,NaN,NaN,NaN


In [141]:
product1 = missing_components[missing_components.smiles_product_1 == "Not found"].product_1.tolist()
product2 = missing_components[missing_components.smiles_product_2 == "Not found"].product_2.tolist()
substrate1 = missing_components[missing_components.smiles_substrate_1 == "Not found"].substrate_1.tolist()
substrate2 = missing_components[missing_components.smiles_substrate_2 == "Not found"].substrate_2.tolist()

with open('../data/protein/cgan/missing_components_for_chosen_class_2.txt', 'w') as f:
    for item in set(product1 +product2 + substrate1 + substrate2):
        f.write("%s\n" % item)

In [142]:
missing_components.to_csv("../data/protein/cgan/missing_components_for_interesting_classes_2.txt", "\t")

In [65]:
missing_components = missing_components[(missing_components.product_3.isna()) & (missing_components.substrate_3.isna())]

In [269]:
final_dataset = proteins_for_joining.drop(smiles_columns, axis= 1)

In [48]:
final_dataset[final_dataset["EC class"].str.startswith("1")].count()

NameError: name 'final_dataset' is not defined

In [43]:
proteins[proteins["EC class"].str.startswith("5.3.1.1")].head(100)

,EC class,Reaction,product_1,product_2,product_3,product_4,product_5,product_6,product_7,product_8,substrate_1,substrate_2,substrate_3,substrate_4,substrate_5
5830,5.3.1.1,D-glyceraldehyde 3-phosphate = glycerone phosphate,glycerone phosphate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D-glyceraldehyde 3-phosphate,NaN,NaN,NaN,NaN
5838,5.3.1.12,D-glucuronate = D-fructuronate,D-fructuronate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D-glucuronate,NaN,NaN,NaN,NaN
5839,5.3.1.13,D-arabinose 5-phosphate = D-ribulose 5-phosphate,D-ribulose 5-phosphate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D-arabinose 5-phosphate,NaN,NaN,NaN,NaN
5840,5.3.1.14,L-rhamnopyranose = L-rhamnulose,L-rhamnulose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L-rhamnopyranose,NaN,NaN,NaN,NaN
5841,5.3.1.15,D-lyxose = D-xylulose,D-xylulose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D-lyxose,NaN,NaN,NaN,NaN
5842,5.3.1.16,1-(5-phospho-beta-D-ribosyl)-5-((5-phospho-beta-D- ribosylamino)methylideneamino)imidazole-4-carboxamide = 5-((5-phospho-1- deoxy-D-ribulos-1-ylamino)methylideneamino)-1-(5-phospho-beta-D- ribosyl)imidazole-4-carboxamide,5-((5-phospho-1- deoxy-D-ribulos-1-ylamino)methylideneamino)-1-(5-phospho-beta-D- ribosyl)imidazole-4-carboxamide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-(5-phospho-beta-D-ribosyl)-5-((5-phospho-beta-D- ribosylamino)methylideneamino)imidazole-4-carboxamide,NaN,NaN,NaN,NaN
5843,5.3.1.17,"5-dehydro-4-deoxy-D-glucuronate = 3-deoxy-D-glycero-2,5-hexodiulosonate","3-deoxy-D-glycero-2,5-hexodiulosonate",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5-dehydro-4-deoxy-D-glucuronate,NaN,NaN,NaN,NaN


In [270]:
final_dataset.to_csv("../data/protein/cgan/enzyme_with_smiles.csv", sep='\t', index=None)

# Sequences with smiles to numpy

In [56]:
import pandas as pd
import numpy as np

from common.bio.smiles import *
from common.bio.amino_acid import *

In [57]:
SMILES_LENGTH=100
SEQUENCE_LENGTH=256

In [453]:
enzyme_with_smiles = pd.read_csv("../data/protein/cganenzyme_with_smiles.csv", sep='\t')
enzyme_with_smiles.head()

,EC class,smiles_product_1,smiles_product_2,smiles_substrate_1,smiles_substrate_2
0,1.1.1.1,*C=O,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,*CO,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
1,1.1.1.4,CC(=O)C(C)O,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,CC(O)C(C)O,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
2,1.1.1.6,O=C(CO)CO,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,OCC(O)CO,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
3,1.1.1.7,CC(=O)COP(=O)(O)O,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,CC(O)COP(=O)(O)O,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
4,1.1.1.8,O=C(CO)COP(=O)(O)O,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,O=P(O)(O)OCC(O)CO,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1


In [456]:
smiles_columns = enzyme_with_smiles.columns.values[1:]
list(smiles_columns)

['smiles_product_1',
 'smiles_product_2',
 'smiles_substrate_1',
 'smiles_substrate_2']

In [458]:
lengths = []
for col in smiles_columns:
    lengths.append(enzyme_with_smiles[col].str.len().max())
max(lengths)

244

In [461]:
enzyme_with_smiles.sort_values("EC class")

,EC class,smiles_product_1,smiles_product_2,smiles_substrate_1,smiles_substrate_2
0,1.1.1.1,*C=O,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,*CO,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
256,1.1.1.10,O=C(CO)C(O)C(O)CO,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)C=CC1,OCC(O)C(O)C(O)CO,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)c1
303,1.1.1.101,CCCCCCCCCCCCCCCC(=O)OCC(=O)COP(=O)(O)O,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)C=CC1,CCCCCCCCCCCCCCCC(=O)OCC(O)COP(=O)([O-])[O-],NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)c1
304,1.1.1.102,CCCCCCCCCCCCCCCC(=O)C(N)CO,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)C=CC1,CCCCCCCCCCCCCCCC(O)C(N)CO,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(OP(=O)(O)O)C3O)C(O)C2O)c1
53,1.1.1.103,CC(=O)C(N)C(=O)[O-],NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,CC(O)C(N)C(=O)O,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
54,1.1.1.104,O=C1CNC(C(=O)O)C1,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,O=C(O)C1CC(O)CN1,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
55,1.1.1.106,CC(C)(C=O)C(O)C(=O)O,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,CC(C)(CO)C(O)C(=O)[O-],NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
56,1.1.1.107,Cc1ncc2c(c1O)C(=O)OC2,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,Cc1ncc(CO)c(C=O)c1O,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
57,1.1.1.108,C[N+](C)(C)CC(=O)CC(=O)[O-],NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,C[N+](C)(C)CC(O)CC(=O)[O-],NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1
6,1.1.1.11,O=C(CO)C(O)C(O)CO,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)C=CC1,OCC(O)C(O)C(O)CO,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1


Filtering out rows that are too huge

In [26]:
filtered_sequences_with_smiles = sequences_with_smiles[sequences_with_smiles.Sequence.str.len()<=SEQUENCE_LENGTH]
filtered_sequences_with_smiles["smiles_product_1"].head()
for col in smiles_columns:
    filtered_sequences_with_smiles = filtered_sequences_with_smiles[filtered_sequences_with_smiles[col].str.len()
                                                                    <= SMILES_LENGTH]

In [28]:
filtered_sequences_with_smiles.shape

(23417, 6)

### Train and validation splits

In [29]:
val_classes = ["1.3.5.2","2.3.3.13", "3.5.1.18","4.1.1.49", "6.1.1.18", "2.7.1.216", "3.5.1.125", "4.1.1.87", "1.1.1.83", 
               "3.6.1.66", "1.15.1.1", "2.7.1.71", "6.3.1.5"]
val_split_classes = ["2.5.1.6", "1.6.1.66", "4.1.1.39", "6.1.1.15","3.5.4.13"]

In [30]:
train = filtered_sequences_with_smiles[~filtered_sequences_with_smiles["EC number"].isin(val_classes)]
train = train[~train["EC number"].isin(val_split_classes)]

In [31]:
train.shape

(21285, 6)

In [32]:
val = filtered_sequences_with_smiles[filtered_sequences_with_smiles["EC number"].isin(val_classes)]

In [33]:
val.shape

(1620, 6)

In [34]:
to_split = filtered_sequences_with_smiles[filtered_sequences_with_smiles["EC number"].isin(val_split_classes)]
to_split = to_split.sample(frac=1)

In [35]:
split_point = int(to_split.shape[0]/2)
add_to_train = to_split.iloc[:split_point, :]
add_to_val = to_split.iloc[split_point:, :]
add_to_train.shape, add_to_val.shape, split_point

((256, 6), (256, 6), 256)

In [36]:
val = val.append(add_to_val)
train = train.append(add_to_train)
train.shape, val.shape

((21541, 6), (1876, 6))

Converting SMILES to IDs

In [48]:
for col in smiles_columns:
    print("Working with Column {}".format(col))
    filtered_sequences_with_smiles[col] = filtered_sequences_with_smiles[col].str.ljust(SMILES_LENGTH, '0')
    filtered_sequences_with_smiles[col] = from_smiles_to_id(filtered_sequences_with_smiles, col)

Working with Column smiles_product_1
Working with Column smiles_product_2
Working with Column smiles_substrate_1
Working with Column smiles_substrate_2


Converting Amino acids to IDs

In [49]:
filtered_sequences_with_smiles.Sequence = filtered_sequences_with_smiles.Sequence.str.ljust(SEQUENCE_LENGTH, '0')
filtered_sequences_with_smiles["Sequence"] = from_amino_acid_to_id(filtered_sequences_with_smiles, "Sequence")

In [55]:
val.groupby("EC number").count()

,Sequence,smiles_product_1,smiles_product_2,smiles_substrate_1,smiles_substrate_2
EC number,,,,,
1.15.1.1,427,427,427,427,427
2.3.3.13,1,1,1,1,1
2.5.1.6,2,2,2,2,2
2.7.1.71,434,434,434,434,434
3.5.4.13,170,170,170,170,170
3.6.1.66,732,732,732,732,732
4.1.1.39,84,84,84,84,84
6.3.1.5,26,26,26,26,26


### Code to find some data for validation

In [12]:
temp = (filtered_sequences_with_smiles.groupby("EC number").size().reset_index(name='counts')
 .sort_values("counts", ascending=False))

In [54]:
#1.3.5.2, 2.3.3.13, 3.5.1.18, 4.1.1.49, 6.1.1.18
#2.5.1.6 	784
#4.1.1.39 	774
#3.6.1.66 	768
#6.1.1.15 	758

In [20]:
path = "../data/protein/cgan/{}/".format("mini_sample")
path

'../data/protein/cgan/mini_sample/'

In [21]:
train

,Sequence,EC number,smiles_product_1,smiles_product_2,smiles_substrate_1,smiles_substrate_2
127,"[11, 9, 5, 1, 18, 8, 18, 5, 13, 6, 16, 12, 2, ...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."
128,"[11, 9, 5, 1, 18, 8, 18, 5, 13, 6, 16, 12, 2, ...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."
129,"[11, 9, 5, 1, 18, 8, 18, 5, 13, 6, 16, 12, 2, ...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."
130,"[11, 9, 5, 1, 18, 8, 18, 10, 13, 6, 16, 12, 2,...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."
131,"[11, 9, 5, 1, 18, 8, 18, 5, 13, 6, 16, 12, 2, ...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."
132,"[11, 9, 17, 1, 8, 8, 14, 10, 13, 6, 10, 12, 15...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."
133,"[11, 15, 18, 1, 18, 8, 18, 5, 13, 6, 16, 12, 1...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."
134,"[11, 9, 16, 1, 18, 8, 10, 10, 13, 6, 10, 12, 1...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."
135,"[11, 9, 5, 6, 18, 18, 18, 5, 13, 6, 16, 12, 2,...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."
136,"[11, 9, 16, 1, 18, 8, 14, 10, 13, 6, 10, 12, 1...",6.3.5.3; 3.5.1.2,"[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 28, 3, 31, 27, 7, 32, 4, 3, 31, 27, 7...","[26, 33, 10, 40, 33, 40, 33, 11, 33, 10, 40, 3...","[27, 19, 21, 26, 21, 21, 3, 19, 27, 4, 26, 21,..."


In [83]:
def save_cgan_data(data, data_type, path):
    np.save(path+"{}_seq.npy".format(data_type), data.Sequence.values)
    for col in smiles_columns:
        np.save(path + "{}_{}.npy".format(data_type, col), data[col].values)

In [84]:
save_cgan_data(train, "train", path)
save_cgan_data(val, "val", path)

In [208]:
val.to_csv("../data/smiles/val_sequences_with_smiles.csv", sep='\t')
train.to_csv("../data/train_sequences_with_smiles.csv", sep='\t')

# Auto Encoder

In [9]:
SMILES_LENGTH = 128

In [45]:
import csv
kegg = pd.read_csv("../data/kegg_smiles.csv",  header=None, names=["smiles"], quotechar="'")

In [46]:
kegg.size

14606

In [47]:
kegg.smiles = kegg.smiles.str.strip()

In [48]:
kegg.head()

,smiles
0,[H+].[OH-]
1,C1=NC2=C(C(=N1)N)N=CN2C3C(C(C(O3)COP(=O)(O)OP(=O)(O)OP(=O)(O)O)O)O
2,C1=CC(=C[N+](=C1)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)O)O)O)O)C(=O)N
3,C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)O)O)O)O
4,C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)OP(=O)(O)O)O)O)O


In [49]:
kegg = kegg[kegg.smiles.str.len() < SMILES_LENGTH]

In [50]:
kegg.size

14191

In [51]:
kegg.smiles = kegg.smiles.str.ljust(SMILES_LENGTH, ' ')

In [52]:
kegg.smiles

0        [H+].[OH-]                                                                                                                      
1        C1=NC2=C(C(=N1)N)N=CN2C3C(C(C(O3)COP(=O)(O)OP(=O)(O)OP(=O)(O)O)O)O                                                              
2        C1=CC(=C[N+](=C1)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)O)O)O)O)C(=O)N                                   
3        C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)O)O)O)O                                         
4        C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)OP(=O)(O)O)O)O)O                                
5        C1=CC(=C[N+](=C1)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)OP(=O)(O)O)O)O)O)C(=O)N                          
6        O=O                                                                                                                             
7        C1=NC2=C(C(=N1)N)N=CN2C3C

In [53]:
set_smiles_characters = set([])
[ set_smiles_characters.update(set(val)) for index, val in kegg.smiles.iteritems() ]
set_smiles_characters = list(sorted(set_smiles_characters))

In [54]:
len(set_smiles_characters)

51

In [55]:
indexToCharacter = {i:set_smiles_characters[i] for i in range (len(set_smiles_characters))}
characterToIndex = {set_smiles_characters[i]:i for i in range (len(set_smiles_characters))}
indexToCharacter, characterToIndex

({0: ' ',
  1: '#',
  2: '(',
  3: ')',
  4: '+',
  5: '-',
  6: '.',
  7: '1',
  8: '2',
  9: '3',
  10: '4',
  11: '5',
  12: '6',
  13: '7',
  14: '8',
  15: '9',
  16: '=',
  17: 'A',
  18: 'B',
  19: 'C',
  20: 'F',
  21: 'G',
  22: 'H',
  23: 'I',
  24: 'K',
  25: 'L',
  26: 'M',
  27: 'N',
  28: 'O',
  29: 'P',
  30: 'R',
  31: 'S',
  32: 'T',
  33: 'W',
  34: 'X',
  35: 'Z',
  36: '[',
  37: ']',
  38: 'a',
  39: 'b',
  40: 'd',
  41: 'e',
  42: 'g',
  43: 'i',
  44: 'l',
  45: 'n',
  46: 'o',
  47: 'r',
  48: 's',
  49: 't',
  50: 'u'},
 {' ': 0,
  '#': 1,
  '(': 2,
  ')': 3,
  '+': 4,
  '-': 5,
  '.': 6,
  '1': 7,
  '2': 8,
  '3': 9,
  '4': 10,
  '5': 11,
  '6': 12,
  '7': 13,
  '8': 14,
  '9': 15,
  '=': 16,
  'A': 17,
  'B': 18,
  'C': 19,
  'F': 20,
  'G': 21,
  'H': 22,
  'I': 23,
  'K': 24,
  'L': 25,
  'M': 26,
  'N': 27,
  'O': 28,
  'P': 29,
  'R': 30,
  'S': 31,
  'T': 32,
  'W': 33,
  'X': 34,
  'Z': 35,
  '[': 36,
  ']': 37,
  'a': 38,
  'b': 39,
  'd': 40,
  'e': 

In [56]:
kegg["smiles_converted"] = [[characterToIndex[char] for char in val ] for index, val in kegg.smiles.iteritems()]

In [57]:
kegg.head()

,smiles,smiles_converted
0,[H+].[OH-],"[36, 22, 4, 37, 6, 36, 28, 22, 5, 37, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,C1=NC2=C(C(=N1)N)N=CN2C3C(C(C(O3)COP(=O)(O)OP(=O)(O)OP(=O)(O)O)O)O,"[19, 7, 16, 27, 19, 8, 16, 19, 2, 19, 2, 16, 27, 7, 3, 27, 3, 27, 16, 19, 27, 8, 19, 9, 19, 2, 19, 2, 19, 2, 28, 9, 3, 19, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 3, 28, 3, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2,C1=CC(=C[N+](=C1)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)O)O)O)O)C(=O)N,"[19, 7, 16, 19, 19, 2, 16, 19, 36, 27, 4, 37, 2, 16, 19, 7, 3, 19, 8, 19, 2, 19, 2, 19, 2, 28, 8, 3, 19, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 19, 19, 9, 19, 2, 19, 2, 19, 2, 28, 9, 3, 27, 10, 19, 16, 27, 19, 11, 16, 19, 10, 27, 16, 19, 27, 16, 19, 11, 27, 3, 28, 3, 28, 3, 28, 3, 28, 3, 19, 2, 16, 28, 3, 27, 0, 0, 0, 0, 0, 0, 0, ...]"
3,C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)O)O)O)O,"[19, 7, 19, 16, 19, 27, 2, 19, 16, 19, 7, 19, 2, 16, 28, 3, 27, 3, 19, 8, 19, 2, 19, 2, 19, 2, 28, 8, 3, 19, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 19, 19, 9, 19, 2, 19, 2, 19, 2, 28, 9, 3, 27, 10, 19, 16, 27, 19, 11, 16, 19, 10, 27, 16, 19, 27, 16, 19, 11, 27, 3, 28, 3, 28, 3, 28, 3, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
4,C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C4N=CN=C5N)OP(=O)(O)O)O)O)O,"[19, 7, 19, 16, 19, 27, 2, 19, 16, 19, 7, 19, 2, 16, 28, 3, 27, 3, 19, 8, 19, 2, 19, 2, 19, 2, 28, 8, 3, 19, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 19, 19, 9, 19, 2, 19, 2, 19, 2, 28, 9, 3, 27, 10, 19, 16, 27, 19, 11, 16, 19, 10, 27, 16, 19, 27, 16, 19, 11, 27, 3, 28, 29, 2, 16, 28, 3, 2, 28, 3, 28, 3, 28, 3, 28, 3, 28, 0, 0, 0, 0, ...]"


In [58]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(kegg, test_size=0.2)

In [59]:
len(train), len(test)

(11352, 2839)

In [61]:
import numpy as np
def to_array(data, features):
    return (np.asarray([ np.asarray(element) for element in data[features].values]))

In [62]:
train_feature_data= to_array(train,'smiles_converted')
val_feature_data = to_array(test,'smiles_converted')

In [63]:
train_feature_data.shape, val_feature_data.shape

((11352, 128), (2839, 128))

In [66]:
np.save("../data/protein/smiles/train_smiles_"+str(SMILES_LENGTH),train_feature_data)
np.save("../data/protein/smiles/val_smiles_"+str(SMILES_LENGTH), val_feature_data)